In [1]:
from preprocessing import *
from sklearn.model_selection import KFold
import argparse
from model import *
from train import test
import torch.optim as optim
import pandas as pd

from MatrixVectorizer import *
import networkx as nx
from typing import Union


In [ ]:
# Check for CUDA (GPU support) and set device accordingly
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # For multi-GPU setups
    # Additional settings for ensuring reproducibility on CUDA
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
else:
    device = torch.device("cpu")
    print("CUDA not available. Using CPU.")

In [2]:
# load csvs as numpy
lr_data_path = '../data/lr_train.csv'
hr_data_path = '../data/hr_train.csv'

lr_train_data = pd.read_csv(lr_data_path, delimiter=',').to_numpy()
hr_train_data = pd.read_csv(hr_data_path, delimiter=',').to_numpy()
lr_train_data[lr_train_data < 0] = 0
np.nan_to_num(lr_train_data, copy=False)

hr_train_data[hr_train_data < 0] = 0
np.nan_to_num(hr_train_data, copy=False)

# map the anti-vectorize function to each row of the lr_train_data

lr_train_data_vectorized = np.array([MatrixVectorizer.anti_vectorize(row, 160) for row in lr_train_data])
hr_train_data_vectorized = np.array([MatrixVectorizer.anti_vectorize(row, 268) for row in hr_train_data])

num_samples = hr_train_data_vectorized.shape[0]
num_samples_list = range(num_samples)
sample_to_index = dict(zip(num_samples_list, hr_train_data_vectorized))

split = int(num_samples * 0.8)

In [3]:
lr_train_data_vectorized = torch.tensor(lr_train_data_vectorized, dtype=torch.float32)
hr_train_data_vectorized = torch.tensor(hr_train_data_vectorized, dtype=torch.float32)

from torch.utils.data import Dataset

class NoisyDataset(Dataset):
    def __init__(self, lr_data, hr_data, noise_level=0.01):
        """
        lr_data: Low resolution data (torch.tensor)
        hr_data: High resolution data (torch.tensor)
        noise_level: Standard deviation of Gaussian noise to be added
        """
        self.lr_data = lr_data
        self.hr_data = hr_data
        self.noise_level = noise_level

    def __len__(self):
        return len(self.lr_data)

    def __getitem__(self, idx):
        lr_sample = self.lr_data[idx]
        hr_sample = self.hr_data[idx]

        # Adding Gaussian noise
        noise = torch.randn(lr_sample.size()) * self.noise_level
        noisy_lr_sample = lr_sample + noise

        # Clipping to ensure values are between 0 and 1
        noisy_lr_sample = torch.clamp(noisy_lr_sample, 0, 1)

        return noisy_lr_sample, hr_sample

train_data = NoisyDataset(lr_train_data_vectorized, hr_train_data_vectorized, noise_level=0.5)

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=True)
sample = next(iter(train_data_loader))
print(sample[0].shape, sample[1].shape)

torch.Size([1, 160, 160]) torch.Size([1, 268, 268])


In [4]:

subjects_adj,subjects_labels = lr_train_data_vectorized[:split], hr_train_data_vectorized[:split]

held_out_subjects_adj,held_out_subjects_labels = lr_train_data_vectorized[split:], hr_train_data_vectorized[split:]

In [5]:
num_splt = 3
epochs = 200
lr = 0.00005 # try [0.0001, 0.0005, 0.00001, 0.00005]
lmbda = 17 # should be around 15-20
lamdba_topo = 1 # should be around 0.5-1.5
lr_dim = 160
hr_dim = 320
hidden_dim = 320 # try smaller and larger - [160-512]
padding = 26
dropout = 0.2 # try [0., 0.1, 0.2, 0.3]
args = argparse.Namespace()
args.epochs = epochs
args.lr = lr
args.lmbda = lmbda
args.lamdba_topo = lamdba_topo
args.lr_dim = lr_dim
args.hr_dim = hr_dim
args.hidden_dim = hidden_dim
args.padding = padding
args.p = dropout


In [6]:
cv = KFold(n_splits=3, random_state=42, shuffle=True)

In [7]:
ks = [0.9, 0.7, 0.6, 0.5]
model = GSRNet(ks, args)

In [8]:
# precompute topological measures for hr_train_data_vectorized
# precomputed_measures = precompute_topological_measures(hr_train_data_vectorized)

In [9]:
import time
criterion = nn.L1Loss()

def train(model, optimizer, subjects_adj,subjects_labels, args): 
  
  all_epochs_loss = []
  no_epochs = args.epochs

  for epoch in range(no_epochs):
    epoch_loss = []
    epoch_error = []
    epoch_topo = []

    model.train()
    for lr, hr in train_data_loader:      
      lr = lr.reshape(160, 160)
      hr = hr.reshape(268, 268)

      model_outputs,net_outs,start_gcn_outs,layer_outs = model(lr)
      model_outputs  = unpad(model_outputs, args.padding)

      padded_hr = pad_HR_adj(hr,args.padding)
      eig_val_hr, U_hr = torch.linalg.eigh(padded_hr, UPLO='U')

      loss = args.lmbda * criterion(net_outs, start_gcn_outs) + criterion(model.layer.weights,U_hr) + criterion(model_outputs, hr) 
      start_time = time.time()

      topo = args.lamdba_topo * compute_topological_MAE_loss(hr, model_outputs)

      error = criterion(model_outputs, hr)
      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      epoch_loss.append(loss.item())
      epoch_error.append(error.item())
      epoch_topo.append(topo.item())
      
  
    model.eval()
    print("Epoch: ",epoch+1, "Loss: ", np.mean(epoch_loss), "Error: ", np.mean(epoch_error),
          "Topo: ", np.mean(epoch_topo))
    all_epochs_loss.append(np.mean(epoch_loss))

In [10]:
# # print(model)
# optimizer = optim.Adam(model.parameters(), lr=args.lr)
# # optimizer = optim.SGD(model.parameters(), lr=args.lr)

# for train_index, test_index in cv.split(subjects_adj):
#     subjects_adj_train = subjects_adj[train_index]  # Get training data 
#     subjects_adj_test = subjects_adj[test_index]   # Get testing data 
#     subjects_ground_truth_train = subjects_labels[train_index]
#     subjects_ground_truth_test = subjects_labels[test_index]

#     train(model, optimizer, subjects_adj_train, subjects_ground_truth_train, args, subjects_adj_test, subjects_ground_truth_test)
    
#     print('Held out test score:')
#     test(model, held_out_subjects_adj, held_out_subjects_labels, args)
#     print('------------------------------')

# Final Model & Kaggle Submission

In [11]:
#final train
final_model = GSRNet(ks, args)
optimizer = optim.Adam(final_model.parameters(), lr=args.lr)

final_model.to(device)
lr_train_data_vectorized.to(device)
hr_train_data_vectorized.to(device)

train(final_model, optimizer, lr_train_data_vectorized, hr_train_data_vectorized, args)

Epoch:  1 Loss:  0.43777106746941985 Error:  0.21379898550981533 Topo:  39.7049633757083
Epoch:  2 Loss:  0.3004957930056635 Error:  0.1823972540344307 Topo:  21.88477148410089
Epoch:  3 Loss:  0.29328333242924626 Error:  0.1786811695305887 Topo:  20.170655073519953
Epoch:  4 Loss:  0.2865609928162512 Error:  0.1766832286190844 Topo:  19.415003867920287
Epoch:  5 Loss:  0.28116676139974306 Error:  0.17528427307477254 Topo:  19.27946554972026
Epoch:  6 Loss:  0.27575906495491187 Error:  0.17429556371922977 Topo:  19.075980597627378
Epoch:  7 Loss:  0.2711637485883907 Error:  0.17383947932791566 Topo:  19.383889797918812
Epoch:  8 Loss:  0.2664204063172826 Error:  0.17273907202803446 Topo:  19.05367251641736
Epoch:  9 Loss:  0.2615383422660257 Error:  0.17154153048278328 Topo:  18.853358879774632
Epoch:  10 Loss:  0.2570333658399696 Error:  0.17061601757646322 Topo:  18.913820906313592
Epoch:  11 Loss:  0.2534014636171078 Error:  0.1698098043481747 Topo:  19.18066168116952
Epoch:  12 Los

In [12]:
#Generate submission 

# load csvs as numpy
test_lr_data_path = '../data/lr_test.csv'

lr_test_data = pd.read_csv(test_lr_data_path, delimiter=',').to_numpy()
print(lr_test_data.shape)
lr_test_data[lr_test_data < 0] = 0
np.nan_to_num(lr_test_data, copy=False)


# map the anti-vectorize function to each row of the lr_train_data

lr_test_data_vectorized = np.array([MatrixVectorizer.anti_vectorize(row, 160) for row in lr_test_data])
print(lr_test_data_vectorized.shape)

(112, 12720)
(112, 160, 160)


In [13]:
final_model.eval()
preds = []
for lr in lr_test_data_vectorized:      
  lr = torch.from_numpy(lr).type(torch.FloatTensor)
  
  model_outputs, _, _, _ = final_model(lr)
  model_outputs  = unpad(model_outputs, args.padding)
  preds.append(MatrixVectorizer.vectorize(model_outputs.detach().numpy()))

print(len(preds), preds[0].shape)
r = np.hstack(preds)
print(r.shape)
meltedDF = r.flatten()

112 (35778,)
(4007136,)


In [14]:
n = meltedDF.shape[0]
df = pd.DataFrame({'ID': np.arange(1, n+1),
                   'Predicted': meltedDF})
df.to_csv('submission.csv', index=False)